Referencias: 

https://medium.com/nanonets/topic-modeling-with-lsa-psla-lda-and-lda2vec-555ff65b0b05

https://medium.com/mlreview/topic-modeling-with-scikit-learn-e80d33668730#.vivglhmhv

---

A nivel de documento, una de las formas más útiles de entender el texto es analizando sus temas/tópicos. 

El proceso de aprendizaje, reconocimiento y extracción de estos temas en una colección de documentos se denomina modelado de temas (topic modeling).

En estas guías exploraremos el modelado de temas a través de LSA y LDA.

Todos los algoritmos de topic modelingse basan en el mismo supuesto básico:
* Cada documento consta de una mezcla de temas y
* Cada tema consiste en una colección de palabras.

En otras palabras, los modelos de temas se basan en la idea de que la semántica de nuestro documento en realidad se rige por algunas variables ocultas o "latentes" que no estamos observando. 

Como resultado, el objetivo de topic modeling es descubrir estas variables latentes (temas) que dan forma al significado de nuestro documento y corpus. 

## LSA
Latent Semantic Analysis, o LSA, es una de las técnicas fundamentales en el modelado de temas. 

La idea central es tomar una matriz de lo que tenemos (documentos y términos) y descomponerla en una matriz separada de documentos y temas y una matriz de temas y términos.

El primer paso es generar nuestra matriz documento-término. 

Dados m documentos y n palabras en nuestro vocabulario, podemos construir una matriz A de tamaño m × n en la que cada fila representa un documento y cada columna representa una palabra. 

En la versión más simple de LSA, cada entrada puede ser simplemente un recuento sin procesar del número de veces que la j-ésima palabra apareció en el i-ésimo documento. 

En la práctica, sin embargo, los recuentos sin procesar no funcionan particularmente bien porque no tienen en cuenta la importancia de cada palabra en el documento. Por ejemplo, la palabra "nuclear" probablemente nos informa más sobre el tema(s) de un documento dado que la palabra "prueba".

En consecuencia, los modelos LSA generalmente reemplazan los recuentos sin procesar en la matriz de términos del documento con una puntuación tf-idf. 

Tf-idf, o término frecuencia de documento de frecuencia inversa, asigna un peso para el término j en el documento i de la siguiente manera:

<img src="img/tf_idf.png" alt="drawing" width="400"/>

Intuitivamente, un término tiene un gran peso cuando aparece con frecuencia en todo el documento pero con poca frecuencia en todo el corpus. 

La palabra "compilar" puede aparecer a menudo en un documento, pero debido a que es probable que sea bastante común en el resto del corpus, no tendrá una alta puntuación tf-idf. 

Sin embargo, si la palabra "gentrificación" aparece a menudo en un documento, porque es más rara en el resto del corpus, tendrá un puntaje tf-idf más alto.

Una vez que tenemos nuestra matriz de términos de documentos A, podemos comenzar a pensar en nuestros temas latentes. 

**Con toda probabilidad, A es muy dispersa (sparse), muy ruidosa y muy redundante en sus muchas dimensiones.**

Como resultado, para encontrar los pocos temas latentes que capturan las relaciones entre las palabras y los documentos, queremos realizar una reducción de dimensionalidad en A.

Esta reducción de dimensionalidad se puede realizar usando SVD truncada. 

SVD, o descomposición de valores singulares, es una técnica en álgebra lineal que factoriza cualquier matriz M en el producto de 3 matrices separadas: M = U * S * V, donde S es una matriz diagonal de los valores singulares de M. 

<img src="img/svd.gif" alt="drawing" width="150"/>

Truncated SVD reduce la dimensionalidad seleccionando solo los t valores singulares más grandes y solo manteniendo las primeras t columnas de U y V. En este caso, t es un hiperparámetro que podemos seleccionar y ajustar para reflejar el número de temas que queremos encontrar.

<img src="img/svd_wikipedia.png" alt="drawing" width="300"/>


Podemos pensarlo como sólo mantener las dimensiones más significativas en nuestro espacio transformado.

<img src="img/svd_graph.png" alt="drawing" width="600"/>


En este caso, U de tamaño (m ⨉ t) emerge como nuestra matriz que relaciona temas y documentos, y V de tamaño (n ⨉ t) se convierte en nuestra matriz dque relaciona temas y términos. 

Tanto en U como en V, las columnas corresponden a uno de nuestros temas t. 

En U, las filas representan vectores de documentos expresados en términos de temas; en V, las filas representan vectores de términos expresados en términos de temas.

Con estos vectores de documentos y vectores de términos, ahora podemos aplicar fácilmente medidas como la similitud de coseno para evaluar:
* la similitud de diferentes documentos
* la similitud de diferentes palabras
* la similitud de términos (o "consultas") y documentos (que se vuelve útil en la recuperación de información, cuando queremos recuperar los pasajes más relevantes en una búsqueda).

LSA es rápido y eficiente de usar, pero tiene algunos problemas:
* resultados poco interpretables (no sabemos cuáles son los temas, y los componentes pueden ser arbitrariamente positivos / negativos)
* Necesidad de un gran conjunto de documentos y vocabulario para obtener resultados precisos
* representación poco eficiente

## LDA

Podemos pensar en dirichlet como una "distribución sobre distribuciones". 

En esencia, responde a la pregunta: "dado este tipo de distribución, ¿cuáles son algunas de las distribuciones de probabilidad reales que es probable que vea?"

Considere el ejemplo de comparar distribuciones de probabilidad de mezclas de temas. 

Digamos que el corpus que estamos viendo tiene documentos de 3 áreas temáticas muy diferentes. 

Si queremos modelar esto, el tipo de distribución que queremos será uno que pese mucho un tema específico, y no le dé mucho peso al resto. 

Si tenemos 3 temas, entonces algunas distribuciones de probabilidad específicas que probablemente veríamos son:
Mezcla X: 90% tema A, 5% tema B, 5% tema C
Mezcla Y: 5% tema A, 90% tema B, 5% tema C
Mezcla Z: 5% tema A, 5% tema B, 90% tema C

Si dibujamos una distribución de probabilidad aleatoria de esta distribución de dirichlet, parametrizada por grandes pesos en un solo tema, probablemente obtendríamos una distribución que se parece mucho a la mezcla X, la mezcla Y o la mezcla Z. 

Sería muy improbable tener una instancia con distribución que es el 33% del tema A, el 33% del tema B y el 33% del tema C.

Eso es esencialmente lo que proporciona una distribución de dirichlet: una forma de muestreo de distribuciones de probabilidad de un tipo específico. 

<img src="img/LDA.png" alt="drawing" width="600"/>


A partir de una distribución de dirichlet Dir (α), extraemos una muestra aleatoria que representa la distribución del tema, o mezcla de temas, de un documento en particular. 

La distribución de este tema es θ. 

Basados en θ, seleccionamos un tema particular Z basado en la distribución.

A continuación, de otra distribución de dirichlet Dir (𝛽), seleccionamos una muestra aleatoria que representa la distribución de palabras del tema Z. 

Esta distribución de palabras es φ. De φ, elegimos la palabra w.

Formalmente, el proceso para generar cada palabra de un documento es el siguiente (tenga en cuenta que este algoritmo usa c en lugar de z para representar el tema):

<img src="img/LDA_algorithm.png" alt="drawing" width="600"/>

En LDA, el conjunto de datos sirve como datos de entrenamiento para la distribución de dirichlet de distribuciones de temas de documentos. 


## NMF

LDA se basa en modelos probabilísticos, mientras que NMF se basa en álgebra lineal. 

Ambos algoritmos toman como entrada una matriz bag of words (es decir, cada documento representado como una fila, con cada columna que contiene el recuento de palabras en el corpus). 

El objetivo de ambos algoritmos es producir 2 matrices más pequeñas; una matriz de documento a tema y una matriz de palabra a tema que cuando se multiplican juntas reproducen la matriz de bolsa de palabras disminuyendo el error.

